In [1]:
import importlib
import cv2
import json
import os
from _paths import nomeroff_net_dir


In [4]:
# auto download latest dataset
from nomeroff_net.tools import modelhub

# auto download latest dataset
#info = modelhub.download_dataset_for_model("EuUa1995")
#PATH_TO_DATASET = info["dataset_path"]
PATH_TO_DATASET = os.path.join(nomeroff_net_dir, "data/dataset/TextDetector/ocr_example")

/usr/local/lib64/python3.8/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


In [9]:
def compare(model_path='latest',
           text_detector_name = "eu_ua_1995",
           img_format = "png",
           root_dir=os.path.join(PATH_TO_DATASET, "test"),
           predicted_part_size=1,
           acc_less_than = 0.7,
           replace_tamplate = {'moderation': {'isModerated': 1, 'moderatedBy': 'ApelSYN'}}):
    text_detector_module = importlib.import_module("nomeroff_net.TextDetectors."+text_detector_name)
    text_detector = getattr(text_detector_module, text_detector_name)
    text_detector.load(model_path)

    ann_dir = os.path.join(root_dir, "ann")
    jsons = []
    jsons_paths = []
    for dir_name, subdir_list, file_list in os.walk(ann_dir):
        for fname in file_list:
            fname = os.path.join(ann_dir, fname)
            jsons_paths.append(fname)
            with open(fname) as jsonF:
                jsonData = json.load(jsonF)
            jsons.append(jsonData)
    print("LOADED {} ANNOTATIONS".format(len(jsons)))

    img_dir = os.path.join(root_dir, "img")
    imgs = []
    for j in jsons:
        img_path =os.path.join(img_dir, "{}.{}".format(j["name"], img_format))
        img = cv2.imread(img_path)
        imgs.append(img)
    print("LOADED {} IMAGES".format(len(imgs)))

    predicted = []
    accs      = []
    N = int(len(imgs) / predicted_part_size) + 1
    for i in range(N):
        part           = i*predicted_part_size
        part_imgs      = imgs[part:part+predicted_part_size]
        predicted_part, net_out_value_part = text_detector.predict(part_imgs, return_acc=True)
        predicted     += predicted_part


        # get accuracy
        if acc_less_than >= 1:
            # not process acc
            accs  += [1 for _predicted in predicted_part]
            continue
        # process accuracy
        acc_part = []
        for _predicted, _net_out_value in zip(predicted_part, net_out_value_part):
            acc = text_detector.get_acc([_net_out_value], [_predicted])
            acc_part.append(acc)
        accs  += acc_part


    print("PREDICTED {} IMAGES".format(len(predicted)))

    err_cnt = 0
    for i in range(len(jsons_paths)):
        json_path      = jsons_paths[i]
        predicted_item = predicted[i]
        jsonData       = jsons[i]
        acc            = accs[i]
        jsonData["moderation"]["predicted"] = predicted_item

        if jsonData["description"].lower() == jsonData["moderation"]["predicted"].lower() and acc > acc_less_than:
            jsonData["moderation"]["isModerated"] = 1
        else:
            print("Predicted '{}' with acc {}, real: '{}' in file {}".format(
                jsonData["moderation"]["predicted"].lower(),
                acc,
                jsonData["description"].lower(),
                json_path))
            err_cnt = err_cnt+1
            jsonData["moderation"]["isModerated"] = 0
        with open(json_path, "w", encoding='utf8') as jsonWF:
            json.dump(jsonData, jsonWF,  ensure_ascii=False)

    print("Error detection count: {}".format(err_cnt))
    print("Accuracy: {}".format(1-err_cnt/len(predicted)))

In [10]:
compare(acc_less_than = 0.75)

LOADED 1037 ANNOTATIONS
LOADED 1037 IMAGES
PREDICTED 1037 IMAGES
Predicted '3765xm' with acc 0.6398438215255737, real: '33765xm' in file /mnt/data/var/www/nomeroff-net/nomeroff_net/tools/../../data/./dataset/TextDetector/EuUa1995/autoriaNumberplateOcrUa-1995-2021-08-25/test/ann/33765XM_8934.json
Predicted '25930oa' with acc 0.8893561363220215, real: '25933oa' in file /mnt/data/var/www/nomeroff-net/nomeroff_net/tools/../../data/./dataset/TextDetector/EuUa1995/autoriaNumberplateOcrUa-1995-2021-08-25/test/ann/25933OA_17106.json
Predicted '06853a' with acc 0.8745327591896057, real: '06853aa' in file /mnt/data/var/www/nomeroff-net/nomeroff_net/tools/../../data/./dataset/TextDetector/EuUa1995/autoriaNumberplateOcrUa-1995-2021-08-25/test/ann/06853AA_44563.json
Predicted '2593kb' with acc 0.7685607075691223, real: '25933kb' in file /mnt/data/var/www/nomeroff-net/nomeroff_net/tools/../../data/./dataset/TextDetector/EuUa1995/autoriaNumberplateOcrUa-1995-2021-08-25/test/ann/25933KB_34225.json
Pre